In [1]:
from common.model import *
import numpy as np
import torch

Temporal Model

In [2]:
model_tm = TemporalModel(
    num_joints_in = 17,
    in_features = 2,
    num_joints_out = 17,
    filter_widths = [3, 3, 3, 3, 3],
    causal =False,
    dropout = 0.25,
    channels = 1024,
    dense = False
)

In [3]:
batch_2d = np.random.rand(10, model_tm.receptive_field(), 17, 2)
inputs_2d = torch.from_numpy(batch_2d.astype('float32'))

In [4]:
predicted_3d_pos = model_tm(inputs_2d)

In [5]:
predicted_3d_pos.shape

torch.Size([10, 1, 17, 3])

convert a batch data [10, 27, 17, 2] to [10, 27, 34] to [10, 34, 27] (batch, channels, frames)

In [6]:
x = inputs_2d
print(x.shape)
sz = x.shape[:3]
x = x.view(x.shape[0], x.shape[1], -1)
print(x.shape)
x = x.permute(0, 2, 1)
print(x.shape)

torch.Size([10, 243, 17, 2])
torch.Size([10, 243, 34])
torch.Size([10, 34, 243])


conv1d layer

In [7]:
print("input : ", x.shape)
print("layer : ", model_tm.expand_conv)
x = model_tm.expand_conv(x)
print("conv1d : ", x.shape)

input :  torch.Size([10, 34, 243])
layer :  Conv1d(34, 1024, kernel_size=(3,), stride=(1,), bias=False)
conv1d :  torch.Size([10, 1024, 241])


bn 1d 

In [8]:
print("layer : ", model_tm.expand_bn)
x = model_tm.expand_bn(x)
print("bn 1d : ", x.shape)

layer :  BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
bn 1d :  torch.Size([10, 1024, 241])


Relu

In [9]:
print("layer : ", model_tm.relu)
x = model_tm.relu(x)
print("relu : ", x.shape)

layer :  ReLU(inplace)
relu :  torch.Size([10, 1024, 241])


drop out 0.25

In [10]:
print("layer : ", model_tm.drop)
x = model_tm.drop(x)
print("drop : ", x.shape)

layer :  Dropout(p=0.25)
drop :  torch.Size([10, 1024, 241])


In [11]:
model_tm.pad

[1, 3, 9, 27, 81]

In [12]:
pad = model_tm.pad[1]

In [13]:
shift = model_tm.causal_shift[1]

In [14]:
res = x[:,:,pad+shift : x.shape[2] - pad+shift]
print("res : ", res.shape)

res :  torch.Size([10, 1024, 235])


In [15]:
print("block 1a : ")
print(model_tm.layers_conv[0])
print(model_tm.layers_bn[0])
print(model_tm.relu)
print(model_tm.drop)
x = model_tm.drop(model_tm.relu(model_tm.layers_bn[0](model_tm.layers_conv[0](x))))
print("block 1a out : ", x.shape)

block 1a : 
Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), dilation=(3,), bias=False)
BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace)
Dropout(p=0.25)
block 1a out :  torch.Size([10, 1024, 235])


In [16]:
print("block 1b : ")
print(model_tm.layers_conv[1])
print(model_tm.layers_bn[1])
print(model_tm.relu)
print(model_tm.drop)
x = model_tm.drop(model_tm.relu(model_tm.layers_bn[1](model_tm.layers_conv[1](x))))
print("block 1b out : ", x.shape)

block 1b : 
Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace)
Dropout(p=0.25)
block 1b out :  torch.Size([10, 1024, 235])
